# requests

Extensive documentation can be found at the [Requests Home Page](http://docs.python-requests.org/en/master/) or [GitHub repository](https://github.com/requests/requests).

In [ ]:
import requests

In [ ]:
import os

## Submitting a GET Request

### A Simple GET Request

Let's start by retrieving the Home Page of [Private Property](https://www.privateproperty.co.za/).

In [ ]:
r = requests.get("https://www.privateproperty.co.za/")

Check if the request was successful.

In [ ]:
r.status_code

Look at the headers in the response.

In [ ]:
r.headers

The web is a wonderful place, filled with all sort of exotic documents. If encoding is specified in the HTML then this is obvious. Otherwise the HTTP headers are used to make an educated guess about the document's encoding.

In [ ]:
r.encoding

If `requests` get the encoding wrong then you can assign the correct value to this attribute.

### A GET Request with Parameters

Let's submit a search request on Private Property.

The search form has a number of fields, some of which are hidden. We can find out about those (and check the default values) by exploring the page contents.

In [ ]:
params = {
    'locationPhrase' : 'Glenwood, Durban',
    'listingType' : 'Sales',
}

In [ ]:
r = requests.get("https://www.privateproperty.co.za/Portal/Search/SearchBoxSearch", params=params)

Let's take a look at the response.

In [ ]:
print(r.text)

Obviously we need some other tools to make sense of that!

### Requesting Binary

You can also access the binary content of the response directly. This is useful, for example, if the document retrieved is an image.

In [ ]:
r = requests.get("https://upload.wikimedia.org/wikipedia/commons/thumb/0/0a/Python.svg/500px-Python.svg.png")
r.content

This can immediately be transformed into an image object.

In [ ]:
from PIL import Image
from io import BytesIO

img = Image.open(BytesIO(r.content))
img.show()

### Requesting JSON

The `json()` method will perform JSON decoding.

In [ ]:
r = requests.get('https://api.github.com/events')
r.text

That's a big JSON document. It'd be easier if it were translated into a Python data type. That's the job of the `json()` method.

In [ ]:
events = r.json()
type(events)

In this case the response has been decoded into a list of dictionaries.

In [ ]:
events

In [ ]:
events[0]['id']

## Submitting a POST Request

We're going to subscribe John Smith to [One Day Only](https://www.onedayonly.co.za/). If you are not yet a subscriber, feel free to substitute your details below. You can always cancel the subscription later.

First we'll submit the data to [HTTPBIN](http://httpbin.org/).

In [ ]:
payload = {
    'firstname': 'John',
    'lastname': 'Smith',
    'email': 'john.smith@gmail.com'
}
r = requests.post("http://httpbin.org/post", data=payload)

In [ ]:
print(r.text)

Now we'll actually submit the form.

In [ ]:
r = requests.post("https://www.onedayonly.co.za/subscribe/campaign/confirm/", data=payload)

There's a voucher code included in the response. It'd be cool to retrieve that. Maybe we'll give it a go later.

### Checking for Success

Check if we were successful.

In [ ]:
r

In [ ]:
r.status_code

In [ ]:
requests.codes

In [ ]:
r.status_code == requests.codes.ok

### Dealing with Failure

Not every request will be successful. We need to gracefully deal with failure.

This will throw an exception if the request was not successful.

In [ ]:
r.raise_for_status()

Let's illustrate for a 404 error.

In [ ]:
status_404 = requests.get('http://httpbin.org/status/404')
status_404.status_code

Unless you actually checked the status code you'd be blissfuly unaware of the problem.

In [ ]:
try:
    status_404.raise_for_status()
except requests.HTTPError as error:
    print('Something went wrong! [%s.]' % str(error))

### Checking Response Headers

The response headers can be accessed as a dictionary.

In [ ]:
r.headers

In [ ]:
r.headers['Content-Encoding']

But it's a special dictionary where the keys are case insensitive (in order to comply with [RFC 7230 Section 3.2](https://tools.ietf.org/html/rfc7230)).

In [ ]:
r.headers.get('content-encoding')

## Using Custom Headers

Sometimes we'd like to manipulate the request headers.

Let's see what the default User Agent string is.

In [ ]:
r = requests.get('http://www.whatsmyua.info/api/v1/ua')
user_agent = r.json()[0]

In [ ]:
user_agent

In [ ]:
user_agent['ua']['rawUa']

Looks like it's `python-requests`. That's kind of a give away that we're scraping. Perhaps we'd like to look more human.

In [ ]:
headers = {
    'User-Agent': 'Lynx/2.8.6rel.5 libwww-FM/2.14'
}

In [ ]:
requests.get('http://www.whatsmyua.info/api/v1/ua', headers=headers).json()

Use [this site](http://www.whoishostingthis.com/tools/user-agent/) to find the User Agent string for your browser. Might also be worthwhile looking at this [catalog](http://www.useragentstring.com/pages/useragentstring.php) of User Agent strings.

## Cookies

Many web sites store information locally as cookies. If you're interested in looking at cookies in your browser, check out the [EditThisCookie](http://www.editthiscookie.com/) extension.

In [ ]:
r = requests.get("http://www.wikipedia.org")
r.cookies.keys()

Cookies are stored in a (mutable) `RequestsCookieJar` object.

In [ ]:
type(r.cookies)

This is essentially a dictionary.

In [ ]:
r.cookies.get_dict()

In [ ]:
r.cookies['GeoIP']

You can build your own cookies and submit them along with a request.

The `Domain` and `Path` of a cookie specify where it is applied. You can find out more in [RFC 2965](https://tools.ietf.org/html/rfc2965).

In [ ]:
jar = requests.cookies.RequestsCookieJar()

jar.set('foo', '1', domain='httpbin.org', path='/cookies')
jar.set('bar', '5', domain='httpbin.org', path='/elsewhere')

r = requests.get('http://httpbin.org/cookies', cookies=jar)
#
# Note that only the cookies for the 'cookies' path are returned.
#
r.text

However these cookies are not persisted across multiple requests.

In [ ]:
r = requests.get('http://httpbin.org/cookies/set/foo/1')
r.text

In [ ]:
requests.get('http://httpbin.org/cookies').text

To get that working you need to use [sessions](http://docs.python-requests.org/en/master/user/advanced/#session-objects).

## Sessions

Sessions allow you to persist information between requests. Specifically:

- share cookies across requests and
- reuse same TCP connection.

We'll perform the same requests as above but this time using a `Session` object.

In [ ]:
s = requests.Session()

In [ ]:
s.get('http://httpbin.org/cookies/set/foo/1')

In [ ]:
r = s.get('http://httpbin.org/cookies')
r.text

The `Session` object has a number of useful properties. For example:

- `cookies`
- `headers`
- `auth`
- `proxies`
- `max_redirects`.

Let's take a look at `headers`.

In [ ]:
s.headers

If we update this then all subsequent requests in the session will use the updated headers.

In [ ]:
s.headers.update({
    'user-agent': 'Lynx/2.8.6rel.5 libwww-FM/2.14'
})

In [ ]:
s.headers

A `Session` object can also be used as a context manager.

In [ ]:
with requests.Session() as s:
    s.get('http://httpbin.org/get')

## Robust Requests

In [ ]:
try:
    # Good practice to apply a timeout, otherwise this could hang indefinitely.
    r = requests.get('http://github.com', timeout=1)
    # Check status.
    r.raise_for_status()
except requests.HTTPError:
    # 404 error, 500 error.
    pass
except requests.ConnectionError:
    # DNS failure, connection refused.
    pass
except requests.Timeout:
    # Request took too long.
    pass
except requests.TooManyRedirects:
    pass